<a href="https://colab.research.google.com/github/vsewall/too_motion-colab-notebooks/blob/main/audio_reactive_video_v1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#With this notebook, you can turn any video into **audio-reactive**.
##how it works: the volume of the sound affects the speed of the video. quiet sound - normal speed, loud sound - the speed is higher. this means that the final video will be shorter than the original one anyway. be sure to pay attention to the output messages in the cell where a list of selected frames is created. try playing with the slider, it affects the dynamics. if there aren't enough frames, you can try slowing down the original video.

---
###telegram: [тоже моушн](https://t.me/too_motion) 
![visitors](https://visitor-badge.glitch.me/badge?page_id=vsewall2motion_audio-reactive-video)

In [ ]:
#@markdown ##use google drive
use_drive = True #@param {type:"boolean"}

# create paths for frames
import os
imgs_path_in = "/content/video_frames"
imgs_path_out = "/content/selected_frames"
paths_to_create = [imgs_path_in, imgs_path_out]
for path in paths_to_create:  
  if not os.path.exists(path):
    os.mkdir(path)

#from IPython.display import clear_output
if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')

In [ ]:
#@markdown ##find your files and copy the paths here

path_to_video = "/content/drive/MyDrive/video.mp4" #@param {type:"string"}
path_to_audio = "/content/drive/MyDrive/music.mp3" #@param {type:"string"}

In [ ]:
#@markdown ##split video by frames

import cv2
vid_fps = round(cv2.VideoCapture(path_to_video).get(cv2.CAP_PROP_FPS))
total_frames = int(cv2.VideoCapture(path_to_video).get(cv2.CAP_PROP_FRAME_COUNT))

print("input video has", vid_fps, "fps and consists of", total_frames, "frames")
print("process of splitting")

# splitting video
!ffmpeg -i $path_to_video /content/video_frames/%7d.png -hide_banner -loglevel error

print("done!")





In [ ]:
#@markdown ##render sound volume
#@markdown the whole track or just a fragment?
audio_fragment = True #@param {type:"boolean"}
#@markdown set the beginning of the audio and the duration of the fragment in seconds

import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
from IPython.display import clear_output

audio_start =  0#@param {type:"number"}
audio_duration =  10#@param {type:"number"}

if os.path.exists('./temp_audio.wav'):
  !rm './temp_audio.wav'

# work with audio
try:
  rate, signal = wavfile.read(path_to_audio)
  signal = np.mean(signal, axis=1)
except:
  import moviepy.editor as mpy
  audio_clip = mpy.AudioFileClip(path_to_audio)
  if audio_fragment:
    audio_clip = audio_clip.subclip(audio_start, audio_start+audio_duration)  
  audio_clip.write_audiofile('./temp_audio.wav', fps=44100, nbytes=2, codec='pcm_s16le')
  rate, signal = wavfile.read('./temp_audio.wav')
  signal = np.mean(signal, axis=1)

clear_output()

signal = np.abs(signal)
duration = signal.shape[0] / rate
frames = int(np.ceil(duration * vid_fps))
samples_per_frame = signal.shape[0] / frames

volume = [] ###

for frame in range(frames):
  start = int(round(frame * samples_per_frame))
  stop = int(round((frame + 1) * samples_per_frame))
  v = np.mean(signal[start:stop], axis=0)
  volume.append(v) ###

###
volume_max = max(volume)
for i in range(frames):
  volume[i] = round(volume[i] / volume_max,3)
###

# visualise the waveform
plt.figure(figsize=(15,3))
plt.plot(volume, color="blue", alpha=.7)
plt.ylim((0, 1))

plt.show()

print("output video will be", frames, "frames long" )




In [ ]:
#@markdown ##create list of selected frames

#import random
import math

# get frames
frames_to_process_names = [f for f in os.listdir(imgs_path_in) if os.path.isfile(os.path.join(imgs_path_in, f))]
frames_to_process_names.sort()

num_frames = len(frames_to_process_names)

out_frames = []

### SPEED MAP ###
speed_multiply = 1 #@param {type:"slider", min:0.05, max:1.2, step:0.05}
my_speed_map = [1,1,2,4,8,9,10,10,10,10]
speed_map = []

for i in my_speed_map:
  i = math.ceil(i * speed_multiply)
  speed_map.append(i)

#print(speed_map)
#speed_map = [1,1,2,4,8,9,10,10,10,10]
speed_num_vars = len(speed_map)


###
frame_count = 0
for i, v in enumerate(volume):
  
  if i > num_frames:
    break

  v = volume[i] # 0...1
  
  # get speed from speed map depending on volume value (that is range 0...1)    
  idx = v * speed_num_vars            ### extend 0...1 to 0...N-1 (N is len of values of speed map)
  idx = math.floor(idx)               ### correct index
  idx = min(idx, speed_num_vars-1)    ###
  speed = speed_map[idx]              ### take speed by index

  frame_count += speed

  if frame_count > num_frames:
    break
  
  out_frames.append(frames_to_process_names[frame_count-1])
###

if len(out_frames) < frames:
  print("WARNING! not enough frames...")
  print("only", len(out_frames), "frames are ready to be copied")
  print("try to decrease speed_multiply or the duration of the audio")
else:
  print(len(out_frames), "frames are ready to be copied")

print(out_frames)


In [ ]:
#@markdown ##copy selected frames

import os
import glob

files = glob.glob(imgs_path_out+'/*')
for f in files:
    os.remove(f)

for i in out_frames:

  print(i)
  img_in = imgs_path_in + "/" + i
  img_out = imgs_path_out + "/" + i

  !cp $img_in -r $img_out

clear_output()
print("done!")

In [ ]:
#@markdown ##create audio-reactive video
download = True #@param {type:"boolean"}

!ffmpeg -y -framerate $vid_fps -pattern_type glob -i $imgs_path_out"/*.png" -c:v libx264  -r $vid_fps -pix_fmt yuv420p /content/temp_video.mp4 -hide_banner -loglevel error
!ffmpeg -y -i /content/temp_video.mp4 -i /content/temp_audio.wav -map 0:v -map 1:a -c:v copy -shortest /content/OUT_audio-reactive-video.mp4 -hide_banner -loglevel error

print("video is ready")

from google.colab import output, files

if download:
  files.download("/content/OUT_audio-reactive-video.mp4")
else:
  print("look for 'OUT_audio-reactive-video.mp4' on the left in the file panel")







